# 01_LangChain/DSpy Setup with Local Ollama Container Instance

This notebook demonstrates how to set up LangChain and DSpy to work with a local Ollama container instance. We will cover the installation of necessary packages, configuration, and connection to the local Ollama instance.

## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install LangChain, DSpy, and other dependencies.

In [1]:
!pip install langchain langchain_community langchain_ollama dspy requests


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 2: Import Packages

Next, we will import the necessary packages for our setup.

In [2]:
import langchain
import dspy
import requests

c:\Users\piete\Repos\my_chat_gpt\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 3: Configure Connection to Local Ollama Instance

We need to configure the connection to our local Ollama container instance. The following code sets up the connection.

In [3]:
OLLAMA_API_URL = "http://localhost:11434/api"

def get_ollama_version():
    response = requests.get(f"{OLLAMA_API_URL}/version")
    if response.status_code == 200:
        return response.json()
    else:
        return None

try:
    ollama_version = get_ollama_version()
    if ollama_version:
        print(f"Connected to Ollama version: {ollama_version}")
    else:
        print("Failed to connect to Ollama instance.")
except requests.exceptions.ConnectionError:
    print("Failed to connect to Ollama instance, is the Docker container running?")

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/version (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000011023AB2660>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Step 4: Minimal Documentation and Instructions

### LangChain
LangChain is a framework for building applications with large language models (LLMs). It provides tools and abstractions to simplify the development process.

### DSpy
DSpy is a data science library that offers various utilities for data manipulation, analysis, and visualization.

### Ollama
Ollama is a containerized environment for running and managing LLMs. It provides an API for interacting with the models.

### Setup Instructions
1. Ensure Docker is installed and running on your machine.
2. Run the Ollama container using the following command:
   ```
   docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
   ```
3. Verify the Ollama instance is running by accessing [http://localhost:11434/api/version](http://localhost:11434/api/version).
4. Follow the steps in this notebook to install packages, import them, and configure the connection to the Ollama instance.

## Step 5: Demonstrate LangChain and DSpy Integration

In this step, we will demonstrate how to use LangChain and DSpy together in a practical example. We will use LangChain to generate text and DSpy to analyze the generated text.

### Example: Text Generation and Analysis

1. Use LangChain to generate text based on a prompt.
2. Use DSpy to analyze the generated text.

#### Generate Text with LangChain
We will use LangChain to generate text based on a given prompt.

In [ ]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="llama3.2")
generated_text = model.invoke("Come up with 10 names for a song about parrots")

generated_text

In [ ]:
dspy.configure(lm=model)

def search_wikipedia(query: str, nr_articles=3) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=nr_articles)
    return [x['text'] for x in results]

In [ ]:
search_wikipedia("Chief executive officer")

### References

- [LangChain Documentation](https://python.langchain.com/docs/)
- [DSpy Documentation](https://dspy.ai/tutorials/rag/)
- [LangChain and DSpy Integration](https://www.reddit.com/r/LangChain/comments/1cqexk6/thoughts_on_dspy/)

## Step 6: Zero Shot DSpy Example

In this step, we will demonstrate a zero-shot example using DSpy. We will use LangChain to generate text based on a prompt and then use DSpy to analyze the generated text without any prior training.

### Example: Zero Shot Text Generation and Analysis

1. Use LangChain to generate text based on a prompt.
2. Use DSpy to analyze the generated text without any prior training.

#### Generate Text with LangChain
We will use LangChain to generate text based on a given prompt.

In [ ]:
prompt_zero_shot = "Describe the impact of artificial intelligence on modern society."
generated_text_zero_shot = model.invoke(prompt_zero_shot)
print(generated_text_zero_shot)

#### Analyze Generated Text with DSpy (Zero Shot)
We will use DSpy to analyze the text generated by LangChain without any prior training.

## Step 7: User Input for Software Project Idea

In this step, we will prompt the user to write a software project idea, send it to the LLM, and display the feedback, summary, and plan.

### Example: User Input and Feedback Loop

1. Prompt the user to write a software project idea.
2. Send the idea to the LLM and display the feedback, summary, and plan.
3. Implement a refinement loop to allow the user to provide additional input and receive updated feedback.

#### Prompt User for Software Project Idea
We will prompt the user to write a software project idea.

In [ ]:
project_idea = input("Please write your software project idea: ")

initial_prompt_template = """
You are an AI assistant. The user has written a software project idea. Please provide feedback on the plan, a summary, and a plan.

Project Idea: {project_idea}
"""

initial_prompt = initial_prompt_template.format(project_idea=project_idea)
feedback_summary_plan = model.invoke(initial_prompt)
print(feedback_summary_plan)

#### Implement Refinement Loop
We will implement a refinement loop to allow the user to provide additional input and receive updated feedback.

In [ ]:
while True:
    additional_input = input("Please provide additional input to refine your project idea (or type 'exit' to finish): ")
    if additional_input.lower() == 'exit':
        break
    
    refinement_prompt_template = """
    You are an AI assistant. The user has provided additional input to refine their software project idea. Please provide updated feedback, summary, and plan.
    
    Project Idea: {project_idea}
    Additional Input: {additional_input}
    """
    
    refinement_prompt = refinement_prompt_template.format(project_idea=project_idea, additional_input=additional_input)
    updated_feedback_summary_plan = model.invoke(refinement_prompt)
    print(updated_feedback_summary_plan)